In [1]:

import pandas as pd
import datetime as datetime
from datetime import date
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

import numpy as np
from datetime import datetime
from datetime import datetime, timedelta


In [188]:
df_merged_all = pd.read_csv('df_hutchison_4h.csv', sep='\t')


In [66]:
df_merged_all = pd.read_csv('df_overnight_and_PRO.csv', sep='\t')


In [67]:
df_merged_all

,Unnamed: 0,id,time,gl,phase,label,axis1,axis2,axis3,steps,lux,inclineOff,inclineStanding,inclineSitting,inclineLying,vm,joinID
0,0,1,2020-06-23 12:38:00+00:00,125.0,screening,Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2020-06-23 12:45:00+00:00,120.0,screening,Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1,2020-06-23 12:45:00+00:00,121.0,screening,Undefined,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1,2020-06-23 12:53:00+00:00,114.0,screening,Undefined,705.0,593.0,1321.0,8.0,0.0,0.0,60.0,0.0,0.0,1611.0,NaN
4,4,1,2020-06-23 13:09:00+00:00,107.0,screening,Undefined,363.0,266.0,578.0,16.0,0.0,0.0,0.0,1.0,59.0,733.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112974,112974,47,2021-12-07 07:06:00+00:00,108.0,lTRE,Undefined,5565.0,5930.0,5865.0,36.0,0.0,0.0,60.0,0.0,0.0,10027.0,NaN
112975,112975,47,2021-12-07 07:21:00+00:00,110.0,lTRE,Undefined,2043.0,2564.0,3787.0,15.0,0.0,0.0,60.0,0.0,0.0,5009.0,NaN
112976,112976,47,2021-12-07 07:36:00+00:00,110.0,lTRE,Undefined,NaN,171.0,112.0,0.0,0.0,0.0,15.0,45.0,0.0,205.0,47.lTRE.2021-12-07
112977,112977,47,2021-12-07 07:51:00+00:00,109.0,lTRE,Undefined,4.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,60.0,4.0,NaN


In [68]:
df_merged_all['time'] = pd.to_datetime(df_merged_all['time'] ,errors = 'coerce')

df_merged_all = df_merged_all.reset_index(drop=True)
df_merged_all = df_merged_all.drop(["Unnamed: 0", "joinID"], axis=1)

df_merged_all = df_merged_all.rename(columns={'label': 'fasting_state'})


fasting_states_to_keep = ['fasting', 'non-fasting']
df_merged_all = df_merged_all[df_merged_all.fasting_state.isin(fasting_states_to_keep)]

final_df = df_merged_all.sort_values(by=['id', 'time'], ascending=True)
final_df['Day'] = final_df["time"].dt.date

## Olga Anfrage

In [191]:
hutch_df = final_df

In [192]:
map_dict = {"non-fasting": 1, "fasting":0}
hutch_df["fasting_state"] = hutch_df["fasting_state"].map(map_dict)


In [193]:
hutch_df['C'] = hutch_df['fasting_state'].diff()

In [194]:
hutch_df['C'] = hutch_df['C'].replace({'0':np.nan, 0:np.nan})

In [196]:
counter_zero = 1
counter_one = 0
last_phase = "screening"

hutch_df["series"] = np.nan

for index, row in hutch_df.iterrows():
    
    if last_phase == row['phase']:
    
        if  not np.isnan(row['C']):

            if row['fasting_state'] == 0:
                counter_zero += 2
                hutch_df.at[index,'series'] = counter_zero


            if row['fasting_state'] == 1:
                counter_one += 2
                hutch_df.at[index,'series'] = counter_one

        elif np.isnan(row['C']):

            if row['fasting_state'] == 0:
                hutch_df.at[index,'series'] = counter_zero

            if row['fasting_state'] == 1:
                hutch_df.at[index,'series'] = counter_one
    else:
            
            if row['fasting_state'] == 0:
                counter_zero += 2
                hutch_df.at[index,'series'] = counter_zero


            if row['fasting_state'] == 1:
                counter_one += 2
                hutch_df.at[index,'series'] = counter_one
                
            last_phase = row['phase']


        
        

In [197]:
hutch_result_df = hutch_df.groupby(["phase", "id","series", "fasting_state"]).agg({'gl': 'mean', 'Day': lambda x: list(set(x))})

In [158]:
hutch_result_df = hutch_df.groupby(["phase", "id","series", "fasting_state"]).agg({'gl': 'mean'})

In [198]:
hutch_result_df = hutch_result_df.reset_index()
hutch_result_df = hutch_result_df.drop('series', axis=1)
hutch_result_df.rename(columns={'gl': 'gl_mean'}, inplace=True)
hutch_result_df = hutch_result_df.sort_values(by=['id', 'phase'], ascending=True)

In [199]:
df_fastingGlucose = hutch_result_df.loc[hutch_result_df['fasting_state'] == 0]
df_postprandialGlucose = hutch_result_df.loc[hutch_result_df['fasting_state'] == 1]

In [200]:
df_fastingGlucose

,phase,id,fasting_state,gl_mean,Day
1,eTRE,1,0,99.700000,[2020-07-21]
3,eTRE,1,0,109.627119,"[2020-07-21, 2020-07-22]"
5,eTRE,1,0,99.189655,"[2020-07-22, 2020-07-23]"
7,eTRE,1,0,96.762712,"[2020-07-24, 2020-07-23]"
9,eTRE,1,0,90.529412,"[2020-07-25, 2020-07-24]"
...,...,...,...,...,...
2091,screening,47,0,85.133333,[2021-10-18]
2092,screening,47,0,86.285714,"[2021-10-18, 2021-10-19]"
2093,screening,47,0,98.969697,"[2021-10-20, 2021-10-19]"
2094,screening,47,0,90.586207,[2021-10-21]


In [201]:
hutch_mean_fasting_glucose_per_phase = hutch_result_df.groupby(["id", "phase", "fasting_state"])["gl_mean"].mean().reset_index(name ='gl_mean_per_phase')


In [202]:
df_meanFastingGlucosePerPhase = hutch_mean_fasting_glucose_per_phase.loc[hutch_mean_fasting_glucose_per_phase['fasting_state'] == 0]
df_meanPostprandialGlucosePerPhase = hutch_mean_fasting_glucose_per_phase.loc[hutch_mean_fasting_glucose_per_phase['fasting_state'] == 1]

In [203]:
hutch_mean_fasting_glucose_per_phase

,id,phase,fasting_state,gl_mean_per_phase
0,1,eTRE,0,97.451187
1,1,eTRE,1,120.345114
2,1,lTRE,0,102.712053
3,1,lTRE,1,108.898256
4,1,screening,0,107.792852
...,...,...,...,...
161,47,eTRE,1,85.361749
162,47,lTRE,0,101.107430
163,47,lTRE,1,109.661121
164,47,screening,0,88.743642


In [204]:
writer = pd.ExcelWriter('hutchisonChronoFast.xlsx', engine='xlsxwriter')

In [205]:
df_fastingGlucose.to_excel(writer, sheet_name='fastingGlucose')
df_meanFastingGlucosePerPhase.to_excel(writer, sheet_name='meanFastingGlucosePerPhase')
df_postprandialGlucose.to_excel(writer, sheet_name='postprandialGlucose')
df_meanPostprandialGlucosePerPhase.to_excel(writer, sheet_name='meanPostprandialGlucosePerPhase')

writer.save()

# für jeden Participant, die mean fasting glucose berechnen



In [69]:
df_screening = final_df[(final_df['phase'] == "screening")]
df_eTRE = final_df[(final_df['phase'] == "eTRE")]
df_lTRE = final_df[(final_df['phase'] == "lTRE")]

In [70]:
mean_fasting_glucose_per_phase_day = final_df.groupby(["id", "phase", "fasting_state", "Day"])["gl"].mean().reset_index(name ='gl_mean')
mean_fasting_glucose_per_phase = final_df.groupby(["id", "phase", "fasting_state"])["gl"].mean().reset_index(name ='gl_mean')


In [71]:
mean_fasting_glucose_per_phase

,id,phase,fasting_state,gl_mean
0,1,eTRE,fasting,96.751678
1,1,eTRE,non-fasting,122.347170
2,1,lTRE,fasting,104.381443
3,1,lTRE,non-fasting,108.125498
4,1,screening,fasting,104.892256
...,...,...,...,...
166,47,eTRE,non-fasting,86.290503
167,47,lTRE,fasting,100.902174
168,47,lTRE,non-fasting,110.036364
169,47,screening,fasting,91.882759


In [72]:
mean_non_fasting_glucose= mean_fasting_glucose_per_phase[mean_fasting_glucose_per_phase.fasting_state != "non-fasting"]

In [73]:
mean_non_fasting_glucose = mean_non_fasting_glucose.drop(["fasting_state"], axis=1)

In [74]:
mean_non_fasting_glucose

,id,phase,gl_mean
0,1,eTRE,96.751678
2,1,lTRE,104.381443
4,1,screening,104.892256
6,2,eTRE,85.075697
8,2,lTRE,87.040134
...,...,...,...
161,46,lTRE,106.088889
163,46,screening,82.625000
165,47,eTRE,74.234375
167,47,lTRE,100.902174


In [75]:
merged_df = pd.merge(final_df, mean_non_fasting_glucose,  on=["id", "phase"])

In [76]:
merged_df

,id,time,gl,phase,fasting_state,axis1,axis2,axis3,steps,lux,inclineOff,inclineStanding,inclineSitting,inclineLying,vm,Day,gl_mean
0,1,2020-06-23 23:09:00+00:00,88.0,screening,fasting,601.0,1059.0,533.0,6.0,0.0,0.0,60.0,0.0,0.0,1330.0,2020-06-23,104.892256
1,1,2020-06-23 23:12:00+00:00,84.0,screening,fasting,853.0,1129.0,718.0,7.0,0.0,0.0,60.0,0.0,0.0,1587.0,2020-06-23,104.892256
2,1,2020-06-23 23:24:00+00:00,91.0,screening,fasting,198.0,241.0,198.0,2.0,0.0,0.0,1.0,22.0,37.0,370.0,2020-06-23,104.892256
3,1,2020-06-23 23:39:00+00:00,88.0,screening,fasting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-23,104.892256
4,1,2020-06-23 23:54:00+00:00,86.0,screening,fasting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-23,104.892256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68000,47,2021-12-07 02:51:00+00:00,112.0,lTRE,fasting,180.0,716.0,255.0,2.0,0.0,40.0,0.0,0.0,20.0,782.0,2021-12-07,100.902174
68001,47,2021-12-07 03:06:00+00:00,107.0,lTRE,fasting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07,100.902174
68002,47,2021-12-07 03:21:00+00:00,103.0,lTRE,fasting,740.0,521.0,251.0,1.0,0.0,0.0,60.0,0.0,0.0,940.0,2021-12-07,100.902174
68003,47,2021-12-07 03:36:00+00:00,102.0,lTRE,fasting,486.0,536.0,260.0,2.0,0.0,0.0,60.0,0.0,0.0,769.0,2021-12-07,100.902174


In [77]:
def compliance_score(x): 
    
    if x['gl'] >= x['gl_mean'] and x['fasting_state'] == "non-fasting":
        return "compliant"
    elif x['gl'] > x['gl_mean'] and x['fasting_state'] == "fasting":
        return "non-compliant"    
    elif x['gl'] <= x['gl_mean'] and x['fasting_state'] == "non-fasting":
        return "non-compliant"
    elif x['gl'] < x['gl_mean'] and x['fasting_state'] == "fasting":
        return "compliant"
   

In [78]:
merged_df["compliance_score"] = merged_df.apply(compliance_score, axis=1)

In [79]:
merged_df_eTRE = merged_df[(merged_df['phase'] == "eTRE")]
merged_df_without_screening = merged_df[(merged_df['phase'] != "screening")]

merged_df_lTRE = merged_df[(merged_df['phase'] == "lTRE")]

In [80]:
merged_df

,id,time,gl,phase,fasting_state,axis1,axis2,axis3,steps,lux,inclineOff,inclineStanding,inclineSitting,inclineLying,vm,Day,gl_mean,compliance_score
0,1,2020-06-23 23:09:00+00:00,88.0,screening,fasting,601.0,1059.0,533.0,6.0,0.0,0.0,60.0,0.0,0.0,1330.0,2020-06-23,104.892256,compliant
1,1,2020-06-23 23:12:00+00:00,84.0,screening,fasting,853.0,1129.0,718.0,7.0,0.0,0.0,60.0,0.0,0.0,1587.0,2020-06-23,104.892256,compliant
2,1,2020-06-23 23:24:00+00:00,91.0,screening,fasting,198.0,241.0,198.0,2.0,0.0,0.0,1.0,22.0,37.0,370.0,2020-06-23,104.892256,compliant
3,1,2020-06-23 23:39:00+00:00,88.0,screening,fasting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-23,104.892256,compliant
4,1,2020-06-23 23:54:00+00:00,86.0,screening,fasting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-23,104.892256,compliant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68000,47,2021-12-07 02:51:00+00:00,112.0,lTRE,fasting,180.0,716.0,255.0,2.0,0.0,40.0,0.0,0.0,20.0,782.0,2021-12-07,100.902174,non-compliant
68001,47,2021-12-07 03:06:00+00:00,107.0,lTRE,fasting,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-12-07,100.902174,non-compliant
68002,47,2021-12-07 03:21:00+00:00,103.0,lTRE,fasting,740.0,521.0,251.0,1.0,0.0,0.0,60.0,0.0,0.0,940.0,2021-12-07,100.902174,non-compliant
68003,47,2021-12-07 03:36:00+00:00,102.0,lTRE,fasting,486.0,536.0,260.0,2.0,0.0,0.0,60.0,0.0,0.0,769.0,2021-12-07,100.902174,non-compliant


In [81]:
result_lTRE = merged_df_lTRE.compliance_score.value_counts()

In [82]:
compliance_score_lTRE = result_lTRE[0] / (result_lTRE[0]+ result_lTRE[1])*100

In [83]:
compliance_score_lTRE

62.693069306930695

In [84]:
result_eTRELTRE = merged_df_without_screening.compliance_score.value_counts()

In [85]:
result_eTRELTRE

compliant        27393
non-compliant    12268
Name: compliance_score, dtype: int64

In [86]:
compliance_score_eTRELTRE = result_eTRELTRE[0] / (result_eTRELTRE[0]+ result_eTRELTRE[1])*100

In [87]:
compliance_score_eTRELTRE

69.06785002899574

In [88]:
result = merged_df.compliance_score.value_counts()

In [89]:
compliance_score = result[0] / (result[0]+ result[1])*100

In [90]:
compliance_score

67.13623998235424

### Accuracy hutchison all


In [91]:
merged_df["fasting_state_pred"] = merged_df.apply(make_prediction_hutchison, axis=1)


In [92]:
result_all = merged_df.compliance_score.value_counts()

In [93]:
compliance_score_all = result_all[0] / (result_all[0]+ result_all[1])*100

In [94]:
compliance_score_all

67.13623998235424

In [95]:
from numpy.lib.stride_tricks import sliding_window_view


def smoothing(y_pred):
    
    outcomes = np.array([])
    last_four_values = y_pred[-4:]
    
    y_pred_windowed = sliding_window_view((y_pred), window_shape = 5)

    i = 0
    for window in y_pred_windowed:
        pred_one = np.count_nonzero(window == 1)
        pred_zero = 5 - pred_one
        if pred_one > pred_zero:
            outcome = int(1)
            outcomes = np.append(outcomes, outcome)
        else:
            outcome = int(0)
            outcomes = np.append(outcomes, outcome)
    
    outcomes = np.append(outcomes, last_four_values)
            
    
    return outcomes

In [96]:
map_dict = {"non-fasting": 1, "fasting":0}
merged_df["fasting_state_pred"] = merged_df["fasting_state_pred"].map(map_dict)


In [97]:
merged_df["fasting_state"] = merged_df["fasting_state"].map(map_dict)


In [98]:
result_all_np = merged_df["fasting_state_pred"]
result_actual = merged_df["fasting_state"]

In [99]:
result_all_np = np.array(result_all_np)
result_actual = np.array(result_actual)

In [101]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(result_actual, result_all_np)

0.6713623998235424

In [102]:
smoothed_score = smoothing(result_all_np)

In [103]:
accuracy_score(result_actual, smoothed_score)

0.6815234173957797

In [22]:
merged_df["pred_fasting_state"] =  merged_df.apply(make_prediction_hutchison, axis=1)

In [26]:
mask= ["id", "time", "phase", "gl", "gl_mean", "fasting_state", "pred_fasting_state", "compliance_score"]

In [27]:
merged_df[mask]

,id,time,phase,gl,gl_mean,fasting_state,pred_fasting_state,compliance_score
0,1,2020-06-24 00:39:00+00:00,screening,87.0,111.320144,fasting,fasting,compliant
1,1,2020-06-24 00:54:00+00:00,screening,102.0,111.320144,fasting,fasting,compliant
2,1,2020-06-24 01:09:00+00:00,screening,83.0,111.320144,fasting,fasting,compliant
3,1,2020-06-24 01:39:00+00:00,screening,42.0,111.320144,fasting,fasting,compliant
4,1,2020-06-24 02:39:00+00:00,screening,57.0,111.320144,fasting,fasting,compliant
...,...,...,...,...,...,...,...,...
89679,47,2021-12-04 23:10:00+00:00,lTRE,111.0,101.002950,fasting,non-fasting,non-compliant
89680,47,2021-12-04 23:27:00+00:00,lTRE,118.0,101.002950,fasting,non-fasting,non-compliant
89681,47,2021-12-04 23:31:00+00:00,lTRE,118.0,101.002950,fasting,non-fasting,non-compliant
89682,47,2021-12-04 23:42:00+00:00,lTRE,121.0,101.002950,fasting,non-fasting,non-compliant
